In [10]:
!pip install autoawq
!pip install nvidia-ml-py3

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=1dec89484a1ff764d1e5439f19bfd63b575983d1fc854a25796555dc8cfb61ee
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3


In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import notebook_login as hfl
import awq
from pynvml import *
import torch
import time

In [4]:
hfl()

In [22]:
def printGPUStats():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    
    print(f"GPU Memory Occupied: {info.used // 1024 ** 2} MB.")

In [21]:
model_id = "0xVolt/Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map={"": 0})
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

ImportError: Loading an AWQ quantized model requires auto-awq library (`pip install autoawq`)